In [ ]:
import importlib
import time
from pathlib import Path


def load_func(module_base, func_name):

    try:
        mod = importlib.import_module(module_base)
        if hasattr(mod, func_name):
            return getattr(mod, func_name)
    except Exception:
        pass

   
    py_file = Path(f"{module_base}.py")
    if py_file.exists():
        ns = {}
        try:
            code = py_file.read_text(encoding="utf-8")
            exec(compile(code, str(py_file), "exec"), ns)
            if func_name in ns and callable(ns[func_name]):
                return ns[func_name]
        except Exception:
            pass


    ipynb_file = Path(f"{module_base}.ipynb")
    if ipynb_file.exists():
        try:
            import nbformat
            nb = nbformat.read(str(ipynb_file), as_version=4)
            ns = {}
            for cell in nb.cells:
                if cell.cell_type == 'code':
                    try:
                        exec(cell.source, ns)
                    except Exception:
                        # ignore errors from unrelated cells
                        pass
            if func_name in ns and callable(ns[func_name]):
                return ns[func_name]
        except Exception:
            pass

    return None


from collections import deque

def fallback_bfs(graph, start, goal):

    if start not in graph or goal not in graph:
        return False, None, 0
    q = deque([start])
    parent = {start: None}
    nodes_visited = 0
    while q:
        node = q.popleft()
        nodes_visited += 1
        if node == goal:
            # reconstruct path
            path = []
            cur = goal
            while cur is not None:
                path.append(cur)
                cur = parent[cur]
            path.reverse()
            return True, path, nodes_visited
        for nbr in graph.get(node, []):
            if nbr not in parent:
                parent[nbr] = node
                q.append(nbr)
    return False, None, nodes_visited


def fallback_dfs(graph, start, goal):

    if start not in graph or goal not in graph:
        return False, None, 0
    stack = [start]
    parent = {start: None}
    visited = set()
    nodes_visited = 0
    while stack:
        node = stack.pop()
        if node in visited:
            continue
        visited.add(node)
        nodes_visited += 1
        if node == goal:

            path = []
            cur = node
            while cur is not None:
                path.append(cur)
                cur = parent.get(cur)
            path.reverse()
            return True, path, nodes_visited
        for nbr in graph.get(node, []):
            if nbr not in visited:
                parent[nbr] = node
                stack.append(nbr)
    return False, None, nodes_visited


def run_search_and_count(fn, graph, source, dest, fallback_impl):

    try:
        try:
            res = fn(graph, source, dest)
        except TypeError:
            res = fn(graph, source)
    except Exception:
        return fallback_impl(graph, source, dest)

    if isinstance(res, tuple):
        if len(res) == 3:
            found, path, nodes = res
            return bool(found), path, int(nodes)
        elif len(res) >= 2:
            found, path = res[0], res[1]
            f_found, f_path, f_nodes = fallback_impl(graph, source, dest)
            return bool(found), path, int(f_nodes)
        
    elif isinstance(res, list) or isinstance(res, set):
        path = list(res)
        f_found, f_path, f_nodes = fallback_impl(graph, source, dest)
        return len(path) > 0, path, int(f_nodes)
    elif isinstance(res, bool):
        f_found, f_path, f_nodes = fallback_impl(graph, source, dest)
        return bool(res), f_path, int(f_nodes)

  
    return fallback_impl(graph, source, dest)


def compare_bfs_dfs(graph, source, dest, show_paths=True):

    bfs_fn = load_func('BFS', 'bfs') or load_func('BFS', 'BFS') or load_func('BFS', 'run_bfs')
    dfs_fn = load_func('DFS', 'dfs') or load_func('DFS', 'DFS') or load_func('DFS', 'run_dfs')

    if bfs_fn is None:
        bfs_fn = fallback_bfs
    if dfs_fn is None:
        dfs_fn = fallback_dfs

    bfs_found, bfs_path_val, bfs_nodes = run_search_and_count(bfs_fn, graph, source, dest, fallback_bfs)
    dfs_found, dfs_path_val, dfs_nodes = run_search_and_count(dfs_fn, graph, source, dest, fallback_dfs)


    print(f"Source: {source}  Destination: {dest}")
    print("BFS: ", "found" if bfs_found else "not found", end='')
    print(f"  nodes_visited={bfs_nodes}")
    if show_paths and bfs_path_val:
        print("  BFS path:", bfs_path_val)

    print("DFS: ", "found" if dfs_found else "not found", end='')
    print(f"  nodes_visited={dfs_nodes}")
    if show_paths and dfs_path_val:
        print("  DFS path:", dfs_path_val)

   
    if bfs_nodes < dfs_nodes:
        print(f"BFS traversed fewer nodes ({bfs_nodes} < {dfs_nodes}) -> BFS is faster by this metric.")
    elif dfs_nodes < bfs_nodes:
        print(f"DFS traversed fewer nodes ({dfs_nodes} < {bfs_nodes}) -> DFS is faster by this metric.")
    else:
        print(f"Both traversed the same number of nodes ({bfs_nodes}).")

    return {
        'bfs_found': bfs_found,
        'bfs_path': bfs_path_val,
        'bfs_nodes': bfs_nodes,
        'dfs_found': dfs_found,
        'dfs_path': dfs_path_val,
        'dfs_nodes': dfs_nodes,
    }

if __name__ == '__main__':
    example_graph = {
        'A': ['B', 'C'],
        'B': ['D','F'],
        'C': ['E'],
        'D': ['F'],
        'E': ['F'],
        'F': ['B']
    }
    print("Running comparison on example graph...")
    compare_bfs_dfs(example_graph, 'A', 'F')


Running comparison on example graph...
Source: A  Destination: F
BFS:  found  nodes_visited=5
  BFS path: ['A', 'B', 'F']
DFS:  found  nodes_visited=4
  DFS path: ['A', 'C', 'E', 'F']
DFS traversed fewer nodes (4 < 5) -> DFS is faster by this metric.
